# Imports

In [10]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import pandas as pd

# Datasets

In [2]:
# Read the dataset csv file
ratings = tfds.load('movielens/latest-small-ratings', split='train')
movies = tfds.load('movielens/latest-small-movies', split='train')

# Select the needed feature
# x['features'] can be done because the dataset are in form of FeaturesDict, check the docs
ratings = ratings.map(lambda x: {
    'user_id': x['user_id'],        
    'movie_title': x['movie_title']
    })
movies = movies.map(lambda x: x['movie_title'])

# Randomize the dataset
tf.random.set_seed(20)
shuffled = ratings.shuffle(100000, seed=20, reshuffle_each_iteration=False)

train = shuffled.take(80000)
test = shuffled.skip(80000).take(20000)

If your dataset are from csv, you can convert that to TF dataset. Below is an example of its implementation.

``` python
# Read the dataset csv file
ratings = pd.read_csv("data/ratings_200k.csv")
movies = pd.read_csv("data/movies_2000.csv")

# Add movie title to ratings dataset
ratings = pd.merge(ratings, movies[['title', 'movieId']], on='movieId')[['userId', 'title']]
ratings['userId'] = ratings['userId'].apply(lambda x: str(x).encode())

# Convert dataframe to tf dataset
ratings = tf.data.Dataset.from_tensor_slices(ratings)
movies = tf.data.Dataset.from_tensor_slices(movies['title'])

# Convert ratings to dict type
ratings = ratings.map(lambda x: {'userId': x[0], 'title': x[1]})
```

In [3]:
# Get the unique user id and movie title
# Alternatively, you can skip this step by passing user ids and movie titles to StringLookup().adapt() but it will be slower
movie_titles = movies.batch(1000)
user_ids = ratings.batch(1000000).map(lambda x: x["user_id"])

uniq_movie_titles = np.unique(np.concatenate(list(movie_titles)))
uniq_user_ids = np.unique(np.concatenate(list(user_ids)))

In [4]:
# Convert the user id and movie title to int representation using StringLookup
# These representation will be used in embedding
movie_titles_vocabulary = tf.keras.layers.StringLookup(vocabulary=uniq_movie_titles, mask_token=None)
user_ids_vocabulary = tf.keras.layers.StringLookup(vocabulary=uniq_user_ids, mask_token=None)

In [5]:
# Create a class based on tfrs.Model
class MovieRecModel(tfrs.Model):
  def __init__(
      self,
      user_model: tf.keras.Model, # Query embedding tower as input
      movie_model: tf.keras.Model, # Candidate embedding tower as input
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Assign the query/user and candidate/movie embedding tower to self
    self.user_model = user_model
    self.movie_model = movie_model

    # Assign the retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.
    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

In [6]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

In [7]:
# Create a retrieval model.
model = MovieRecModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Cache the dataset
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=10)

Epoch 1/10
25/25 [==============================] - 43s 2s/step - factorized_top_k/top_1_categorical_accuracy: 3.9668e-05 - factorized_top_k/top_5_categorical_accuracy: 6.2478e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0017 - factorized_top_k/top_50_categorical_accuracy: 0.0182 - factorized_top_k/top_100_categorical_accuracy: 0.0399 - loss: 33377.0575 - regularization_loss: 0.0000e+00 - total_loss: 33377.0575
Epoch 2/10
25/25 [==============================] - 46s 2s/step - factorized_top_k/top_1_categorical_accuracy: 4.9585e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0020 - factorized_top_k/top_10_categorical_accuracy: 0.0071 - factorized_top_k/top_50_categorical_accuracy: 0.0645 - factorized_top_k/top_100_categorical_accuracy: 0.1291 - loss: 30648.4655 - regularization_loss: 0.0000e+00 - total_loss: 30648.4655
Epoch 3/10
25/25 [==============================] - 43s 2s/step - factorized_top_k/top_1_categorical_accuracy: 7.0411e-04 - factorized_top_k/top_5_categoric

In [8]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 8s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0039 - factorized_top_k/top_5_categorical_accuracy: 0.0285 - factorized_top_k/top_10_categorical_accuracy: 0.0593 - factorized_top_k/top_50_categorical_accuracy: 0.2397 - factorized_top_k/top_100_categorical_accuracy: 0.3711 - loss: 26031.1068 - regularization_loss: 0.0000e+00 - total_loss: 26031.1068


{'factorized_top_k/top_1_categorical_accuracy': 0.0038999998942017555,
 'factorized_top_k/top_5_categorical_accuracy': 0.02850000001490116,
 'factorized_top_k/top_10_categorical_accuracy': 0.05925000086426735,
 'factorized_top_k/top_50_categorical_accuracy': 0.2397499978542328,
 'factorized_top_k/top_100_categorical_accuracy': 0.3711499869823456,
 'loss': 23582.830078125,
 'regularization_loss': 0,
 'total_loss': 23582.830078125}

In [11]:
# Use brute-force search to set up retrieval using the trained representations
# This index is used for brute-force retrieval based on user representations
# The model.user_model refers to the trained model that provides the user representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

# .index_from_dataset takes candidates as input which in a form of (movie title, movie title embedding) pairs
# The embedding can be obtained using model.movie_model
index.index_from_dataset(movies.batch(100).map(lambda movie_title: (movie_title, model.movie_model(movie_title))))

In [16]:
# Get 10 recommendations as the result of retrieval stage
scores, retrieved_titles = index(np.array(["20"]), k=10)
print(f"Top 10 recommendations for user 20:")
for movie in retrieved_titles[0]:
    print(f"- {movie.numpy().decode('utf-8')}")

Top 10 recommendations for user 20:
- Adanggaman (2000)
- Black Stallion, The (1979)
- Soft Fruit (1999)
- Shiloh (1997)
- Trail of the Pink Panther (1982)
- Endurance: Shackleton's Legendary Antarctic Expedition, The (2000)
- Jimmy Neutron: Boy Genius (2001)
- Return of Jafar, The (1994)
- Kiss Me Kate (1953)
- Unfaithful (2002)


In [21]:
import os

path = os.path.join(os.curdir, "retrieval_model")

# Save the model query model
tf.saved_model.save(index, path)

INFO:tensorflow:Assets written to: .\retrieval_model\assets


INFO:tensorflow:Assets written to: .\retrieval_model\assets
